In [20]:
# https://www.kaggle.com/kernels/scriptcontent/11511967/notebook

import numpy as np
import pandas as pd
# from IPython.display import display
from tqdm import tqdm
import stanfordnlp
import string

import re
from pprint import pprint
import Levenshtein as lev
from fuzzywuzzy import process,fuzz

import warnings
warnings.filterwarnings("ignore")

# from bokeh.plotting import figure, output_file, show
# from bokeh.models import Label
# from bokeh.io import output_notebook
# output_notebook()

%matplotlib inline

In [4]:
nlp = stanfordnlp.Pipeline ( lang = 'hi' ) 

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\Sarthak\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Cannot load model from C:\Users\Sarthak\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tokenizer.pt


SystemExit: 1

In [5]:
# stanfordNLP initialization

config = {
    "processors" : "tokenize,mwt,lemma,pos",
    "lang" : "hi",
    # Processor-specific arguments are set with keys "{processor_name}_{argument_name}"
    'tokenize_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tokenizer.pt', 
# 	'mwt_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\',
	'pos_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_tagger.pt',
	'pos_pretrain_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb.pretrain.pt',
	'lemma_model_path': 'D:\Data\stanfordnlp_resources\hi_hdtb_models\hi_hdtb_lemmatizer.pt'
# 	'depparse_model_path': './fr_gsd_models/fr_gsd_parser.pt',
# 	'depparse_pretrain_path': './fr_gsd_models/fr_gsd.pretrain.pt'
}

nlp = stanfordnlp.Pipeline(**config) # Initialize the pipeline using a configuration dict
# nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,lemma,pos")

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_lemmatizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: pos
With settings: 
{'model_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_tagger.pt', 'pretrain_path': 'D:\\Data\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb.pretrain.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Done loading processors!
---


In [6]:
datafile = '../data/hindi.txt'
with open(datafile,'r',encoding='utf-8') as f:
    text = f.read()
    text = text.split("॥")

In [12]:
text_sentence = []
i = 0
for word in tqdm(text):
#     print(i,word)
    if re.findall("^\s?\d+\s?",word):
        continue
    else:   
        text_sentence.append(word)

100%|██████████████████████████████████████████████████████████████████████████| 1290/1290 [00:00<00:00, 433476.38it/s]


In [13]:
# hindi stopwords
stop_words_df = pd.read_csv("../data/stopwords.txt", header = None)
stop_words = list(set(stop_words_df.values.reshape(1,-1).tolist()[0]))
stop_words.extend(["।", "।।", ")", "(", ",",'"',"हे", "हो", 'में','से','COMMA'])

In [14]:
def lemmatization(text):
    lemmatized_text = []
    for line in tqdm(text):
        if line not in [""," "] :
            doc = nlp(line)
            for sent in doc.sentences:
                for wrd in sent.words:
                    #extract text and lemma
                    lemmatized_text.append(wrd.lemma)
    return lemmatized_text

In [15]:
lemmatized = lemmatization(text_sentence)

100%|████████████████████████████████████████████████████████████████████████████████| 667/667 [01:10<00:00,  9.48it/s]


In [16]:
def remove_stopwords(word_tokenized,stop_words):
    return [word for word in word_tokenized if word not in stop_words]

In [17]:
clean_text = remove_stopwords(lemmatized,stop_words)
len(clean_text),len(lemmatized)

(7417, 14489)

In [18]:
clean_text[0:5]

['वानर', 'हनुमान', 'सीमा', 'उत्कण्ठा', 'येसूयास्त']

In [21]:
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in clean_text]
print(stripped[:25])
print(len(clean_text),len(stripped))

['वानर', 'हनुमान', 'सीमा', 'उत्कण्ठा', 'येसूयास्त', 'प्रतीक्षा', 'भया', 'घ४९', 'सीता', 'रावण', 'भन्तापुर', 'ढूँढ', '', 'महावीर्य', 'महान', 'हृदय', 'पाछ', 'वानर', 'श्रेष्ठ', 'रात', 'अद्वार', 'कोट', 'फांद', 'प्रविष्ट', 'हृ']
7417 7417


In [22]:
def custom_remove_garbage(original_words_list,list_of_garbage_words):
    tmp_list = [word for word in original_words_list if word not in list_of_garbage_words] # garbage list
    tmp_list = [word for word in tmp_list if len(re.findall("\d+",word))==0] # english numbers
    tmp_list = [word for word in tmp_list if len(re.findall("[a-zA-Z]+",word))==0] # english alphabets
    return tmp_list

In [23]:
clean_text == stripped

# Python code to get difference of two lists 
# Not using set() 
def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

resultant = Diff(clean_text,stripped)
len(set(resultant))

167

In [24]:
final_text = custom_remove_garbage(stripped,resultant)
len(final_text)

7118

In [25]:
final_text = custom_remove_garbage(stripped,resultant)
len(final_text)

7118

In [26]:
final_text[0:5]

['वानर', 'हनुमान', 'सीमा', 'उत्कण्ठा', 'येसूयास्त']

# Fuzzy search

In [27]:
import codecs, difflib, Levenshtein
import jellyfish,textdistance


str_match ='हनुमान'
distance_df = pd.DataFrame(columns=["word","diffl","Levenshtein","Sorensen","Jaccard","fuzz_ratio","partial_ratio",
                                    "token_sort_ratio","token_set_ratio"])
for word in tqdm(final_text):
    diffl   = difflib.SequenceMatcher(None, str_match, word).ratio()
    lev     = Levenshtein.ratio(str_match, word) 
    sor     =  textdistance.sorensen(str_match, word)
    jac     =  textdistance.jaccard(str_match, word)
    fuzz_ratio = fuzz.ratio(str_match,word)/100 
    partial_ratio = fuzz.partial_ratio(str_match,word)/100
    token_sort_ratio = fuzz.token_sort_ratio(str_match,word)/100
    token_set_ratio = fuzz.token_set_ratio(str_match,word)/100
    
    
    distance_list = [word,diffl,lev,sor,jac,fuzz_ratio,partial_ratio,token_sort_ratio,token_set_ratio]
    distance_series = pd.Series(dict(zip(distance_df.columns, distance_list)))
    
    distance_df = distance_df.append(distance_series, ignore_index=True)
#     print( diffl, lev, sor, jac)

100%|█████████████████████████████████████████████████████████████████████████████| 7118/7118 [00:20<00:00, 343.52it/s]


In [28]:
distance_df[distance_df["Levenshtein"]>0.80]

,word,diffl,Levenshtein,Sorensen,Jaccard,fuzz_ratio,partial_ratio,token_sort_ratio,token_set_ratio
1,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
76,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
90,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
330,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
417,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
425,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
446,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
468,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
582,हनुमान्,0.923077,0.923077,0.923077,0.857143,0.92,1.00,1.00,1.00
806,हनुमान,1.000000,1.000000,1.000000,1.000000,1.00,1.00,1.00,1.00
